# microgpt-rs-lab: Colab GPU Benchmark

This notebook runs the GPU benchmark harness (`scripts/benchmark_karpathy.py`) in Google Colab.

Recommended runtime:
- Runtime -> Change runtime type -> GPU
- For faster results, use `PROFILE=quick` (default).


In [ ]:
import os

REPO_URL = os.environ.get("REPO_URL", "https://github.com/ricardofrantz/microgpt-rs-lab.git")
REPO_BRANCH = os.environ.get("REPO_BRANCH", "master")
BASELINE = os.environ.get("BASELINE", "nanogpt")  # nanogpt | nanochat | both
PROFILE = os.environ.get("PROFILE", "quick")      # quick | full
EXTRA_ARGS = os.environ.get("EXTRA_ARGS", "")

print({
    "REPO_URL": REPO_URL,
    "REPO_BRANCH": REPO_BRANCH,
    "BASELINE": BASELINE,
    "PROFILE": PROFILE,
    "EXTRA_ARGS": EXTRA_ARGS,
})


In [ ]:
!nvidia-smi


In [ ]:
import pathlib
import shutil
import subprocess

repo_dir = pathlib.Path("/content/microgpt-rs-lab")
if repo_dir.exists():
    shutil.rmtree(repo_dir)

subprocess.run([
    "git", "clone", "--depth", "1", "--branch", REPO_BRANCH,
    REPO_URL, str(repo_dir)
], check=True)

%cd /content/microgpt-rs-lab
!git rev-parse --short HEAD


In [ ]:
import os
import subprocess

env = os.environ.copy()
env["BASELINE"] = BASELINE
env["PROFILE"] = PROFILE
env["EXTRA_ARGS"] = EXTRA_ARGS

subprocess.run(["bash", "scripts/colab_gpu_benchmark.sh"], check=True, env=env)


In [ ]:
from pathlib import Path

result_files = sorted(Path("results").glob("benchmark_karpathy_*.md"))
if not result_files:
    raise RuntimeError("No benchmark markdown artifact found in results/")

latest = result_files[-1]
print(f"Latest summary: {latest}")
print()
print(latest.read_text())


In [ ]:
# Optional: download all benchmark artifacts
import datetime as dt
import tarfile

archive = f"/content/benchmark_results_{dt.datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}.tar.gz"
with tarfile.open(archive, "w:gz") as tar:
    tar.add("results", arcname="results")

try:
    from google.colab import files
    files.download(archive)
except Exception as exc:
    print(f"Archive created at: {archive}")
    print(f"Download helper unavailable: {exc}")
